In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets
!pip install rouge_score
!pip install transformers
!pip install transformers[torch]
!pip install accelerate
!pip install wandb
!pip install evaluate

In [ ]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer, DataCollatorForSeq2Seq,Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset, Dataset, load_metric
from rouge_score import rouge_scorer
from torch.optim import AdamW
import evaluate
import numpy as np
import gc
import os

In [ ]:
train_data = load_dataset("allenai/multinews_dense_oracle")['train']
val_data = load_dataset("allenai/multinews_dense_oracle")['validation']
test_data = load_dataset("allenai/multinews_dense_oracle")['test']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

In [ ]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    model_inputs = tokenizer(examples['document'], max_length=1024, padding = 'max_length',truncation=True,add_special_tokens=True,  # Ensure special tokens are added
        return_attention_mask=True, return_tensors = 'pt')

    labels = tokenizer(text_target=examples["summary"], max_length=128, padding = 'max_length',truncation=True,add_special_tokens=True,  # Ensure special tokens are added
        return_attention_mask=True, return_tensors = 'pt')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_train_dataset = train_data.map(tokenize_function, batched=True)
tokenized_val_dataset = val_data.map(tokenize_function, batched=True)
tokenized_test_dataset = test_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/899 [00:00<?, ? examples/s]

Map:   0%|          | 0/112 [00:00<?, ? examples/s]

Map:   0%|          | 0/5622 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="facebook/bart-large-cnn")

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/Checkpoints/Run1",
    learning_rate = 3.2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    num_train_epochs=2,
    save_total_limit = 1,
    weight_decay=0.1,
    report_to="wandb"
)

In [ ]:
training_args.set_optimizer(name="adamw_torch")

Seq2SeqTrainingArguments(output_dir='/content/drive/MyDrive/Checkpoints/Run1', overwrite_output_dir=False, do_train=False, do_eval=True, do_predict=False, evaluation_strategy=<IntervalStrategy.EPOCH: 'epoch'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, eval_delay=0, learning_rate=5e-05, weight_decay=0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=2, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, lr_scheduler_kwargs={}, warmup_ratio=0.0, warmup_steps=0, log_level='passive', log_level_replica='warning', log_on_each_node=True, logging_dir='/content/drive/MyDrive/Checkpoints/Run1/runs/Apr24_13-41-54_f2033445a3ae', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=500, logging_nan_inf_filter=True, save_strategy=<IntervalS

In [ ]:
gc.collect()

torch.cuda.empty_cache()

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:20000"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    data_collator=data_collator,
)

In [ ]:
trainer.train(resume_from_checkpoint="/content/drive/MyDrive/Checkpoints/Run1/checkpoint-11244")

You are resuming training from a checkpoint trained with 4.38.2 of Transformers but your current version is 4.40.0. This is not recommended and could yield to errors or unwanted behaviors.
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=11244, training_loss=0.0, metrics={'train_runtime': 8.5419, 'train_samples_per_second': 210.491, 'train_steps_per_second': 26.458, 'total_flos': 1.9491805633236173e+17, 'train_loss': 0.0, 'epoch': 2.0})

In [ ]:
predictions = trainer.predict(tokenized_test_dataset)


In [ ]:
trainer.evaluate(eval_dataset=tokenized_test_dataset)

KeyboardInterrupt: 

In [ ]:
predicted_sentences = []
for prediction in predictions.predictions:
    predicted_sentences.append(tokenizer.decode(prediction, skip_special_tokens=True))

In [ ]:
rouge = evaluate.load('rouge')
references = test_data['summary']
scores = rouge.compute(predictions=predicted_sentences, references=references, use_stemmer=True)
print(f"Rouge Scores: {scores}")

Rouge Scores: {'rouge1': 0.39775850158169634, 'rouge2': 0.15894207996695298, 'rougeL': 0.21928674089274489, 'rougeLsum': 0.2192865757655391}


In [ ]:
metric = load_metric("rouge")
results = metric.compute(predictions=predicted_sentences, references=references)

precision = results["rouge1"].mid.precision
recall = results["rouge1"].mid.recall
f1_score = results["rouge1"].mid.fmeasure

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)

<ipython-input-18-550ec988c433>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Precision: 0.6010330341918408
Recall: 0.2933664252955991
F1 Score: 0.38486487921580326


In [ ]:
print(results)

{'rouge1': AggregateScore(low=Score(precision=0.5973018894745247, recall=0.29114890065138815, fmeasure=0.38239856291287544), mid=Score(precision=0.6010330341918408, recall=0.2933664252955991, fmeasure=0.38486487921580326), high=Score(precision=0.6044816045657883, recall=0.2957839614098866, fmeasure=0.38727687095551516)), 'rouge2': AggregateScore(low=Score(precision=0.23728449349108613, recall=0.11619660846982452, fmeasure=0.1524500674557746), mid=Score(precision=0.24119487948281387, recall=0.11832065586490567, fmeasure=0.15497744334990612), high=Score(precision=0.24546883626287852, recall=0.1205460074537879, fmeasure=0.15777556094816084)), 'rougeL': AggregateScore(low=Score(precision=0.3306118793485785, recall=0.16201532193942794, fmeasure=0.2124270294633342), mid=Score(precision=0.3341375873880205, recall=0.16443161181096777, fmeasure=0.2150165525247949), high=Score(precision=0.3376172523982179, recall=0.1665657775767352, fmeasure=0.21737118139115708)), 'rougeLsum': AggregateScore(low

In [ ]:
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predicted_sentences, references=references)
print(results)

{'bleu': 0.06108490900245825, 'precisions': [0.6144275256314274, 0.24665798638544242, 0.13351880490410217, 0.09274494536749849], 'brevity_penalty': 0.2934839113609561, 'length_ratio': 0.44924992963692656, 'translation_length': 638474, 'reference_length': 1421200}


In [ ]:
%%capture cap
f = open("/content/drive/MyDrive/Checkpoints/Run1/results.txt", "w")
print(f"Rouge Scores: {scores}",file=f)
print(f"Precision: {precision}",file=f)
print(f"Recall: {recall}",file=f)
print(f"F1 Score: {f1_score}",file=f)

In [ ]:
from transformers import pipeline
doc = "When Mount Ruang in Indonesia underwent multiple explosive eruptions last week, volcanic gases were flung so high they reached the atmosphere’s second layer, tens of thousands of feet above ground.Ruangs eruptions ejected a massive ash plume and sent some volcanic gases more than 65,000 feet into the air, according to satellite estimates – about 25,000 feet higher than a commercial airplane will typically fly.The eruption’s potential impacts to weather and climate are starting to come into focus, even as the danger posed by the volcano persists and evacuations continue.It’s possible for volcanoes to have a short-term impact on the climate – including global temperature cooling – due to the gases they inject high into the upper atmosphere. But Mount Ruang’s influence on the climate will likely be minimal, according to Greg Huey, the chair of Georgia Tech’s School of Earth and Atmospheric Sciences.And the day-to-day weather conditions near Mount Ruang – things like temperature, clouds and rain – probably won’t be influenced by the volcano for long, Huey told CNN.Mount Ruang, a 2,400-foot (725-meter) stratovolcano on Ruang Island in Indonesia’s North Sulawesi province, has erupted at least seven times since Tuesday night, the country’s volcanology agency said. Stratovolcanoes can produce explosive eruptions because their cone shape allows gas to build up, according to volcanologists.Volcanic ash is typically a mixture of crushed-up solids – including rocks, minerals and glass – and gases, such as water vapor, carbon dioxide and sulfur dioxide, according to NASA.Crushed-up solids generate a lot of static electricity within ash plumes as they bash into each other, resulting in intense displays of lighting, according to Huey.The ash itself is short-lived in the atmosphere because it’s heavy, it’s big and it tends to settle out quickly,” Huey told CNN. It’s the gases that are able to reach much higher in the atmosphere.Dense ash near the surface creates hazardous air quality and causes a temporary cooling effect as it blocks out warming sunlight. Once the active eruption stops, ash starts to settle."
model2 = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/Checkpoints/Run1/checkpoint-11244')

text_generator = pipeline("summarization", model=model2, tokenizer=tokenizer)
generated_text = text_generator(doc, max_length=128, num_return_sequences=1, do_sample=True)[0]
print(generated_text)